<summary>Table of Contents</summary>

- [1. TimeLLM](#1-timellm)

Results for TimeLLM with input lengths from PatchTST.

In [1]:
import os
import pandas as pd
import subprocess
import time
import shutil
from utils.helper import extract_metrics_from_output, convert_results_into_df, running_time

In [2]:
# Dynamic variables
pred_lens = [24, 96, 168]
countries = ['DE', 'GB', 'ES', 'FR', 'IT']
num_cols = [5, 5, 3, 3, 3]
seq_lens = [512, 512, 336, 168, 168]

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

# 1. TimeLLM

In [3]:
log_dir = f"logs/timellm/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Dynamic variables
model = "TimeLLM"

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}.log"

# Parameters for tuning
lr = 0.001 # 10^-3 
train_epochs = 20
patience = 5
d_model = 16
d_ff = 64
batch_size = 32

# List to store the results
timellm_results = []

In [ ]:
# For single GPU use following command:
# python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --num_machines 1 --dynamo_backend "no" --main_process_port "01025" ./Time-LLM/run_main.py \

start = time.time()

# Open log file
with open(log_file_path, "w") as log_file:
    
    for i, country in enumerate(countries):
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:
            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2)
            
            if country == 'DE' and pred_len == 24:
                seq_len=336
            else:
                seq_len = seq_lens[i]
            
            if country == "FR" and pred_len == 24:
                train_epochs = 30

            # Command to run script with parameters
            command = f"""
            python -m accelerate.commands.launch --mixed_precision bf16 --multi_gpu --num_processes=4 --num_machines 1 --dynamo_backend "no"  --main_process_port "01025" ./Time-LLM/run_main.py \
              --task_name long_term_forecast \
              --is_training 1 \
              --root_path ./datasets/ \
              --data_path {country}_data.csv \
              --model_id {i+1} \
              --model {model} \
              --data {country} \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --factor 3 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --itr 1 \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --batch_size {batch_size} \
              --learning_rate {lr} \
              --llm_model "GPT2" \
              --llm_dim 768 \
              --llm_layers 12 \
              --train_epochs {train_epochs} \
              --patience {patience} \
              --model_comment {model}+{country}
            """

            # Run command and log output
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture and log output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')
                log_file.write(line)

            process.wait()  # Wait for process to finish

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr=1)[0]
            mse, rmse, mae, _ = iteration_metrics
            timellm_results.append({
                'Country': country,
                'Pred_len': pred_len,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae
                })

            # Time tracking for pred_len
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = f"Intermediate time for {country} and pred_len {pred_len}: {hours_int:0>2}h:{mins_int:0>2}m:{secs_int:05.2f}s\n"
            log_file.write(statement_3)
            print(statement_3)

        # Time tracking for each country
        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = f"Intermediate time for {country}: {hours_c:0>2}h:{mins_c:0>2}m:{secs_c:05.2f}s\n"
        log_file.write(statement_4)
        print(statement_4)

    # Total time
    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = f"Total time: {hours:0>2}h:{mins:0>2}m:{secs:05.2f}s\n"
    log_file.write(statement_5)
    print(statement_5)



=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

[2024-11-13 09:49:23,155] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-13 09:49:23,160] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-13 09:49:23,165] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-13 09:49:23,168] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-13 09:49:23,872] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-13 09:49:23,872] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-13 09:49:23,872] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-13 09:49:23,872] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-13 09:49:23,872] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
train 143885

In [ ]:
#shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/timellm'

if not os.path.exists(path):
    os.makedirs(path)

timellm_df = convert_results_into_df(timellm_results, if_loss_fnc=False, itr=1)

# Final DF
timellm_df.columns = pd.MultiIndex.from_product([['TimeLLM'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
timellm_df.to_csv(os.path.join(path, 'timellm.csv'))
timellm_df.round(4)

Model            TimeLLM                
Metrics              MSE    RMSE     MAE
Country Pred_len                        
DE      24        0.0215  0.1468  0.0962
        96        0.0347  0.1861  0.1281
        168       0.0375  0.1936  0.1386
GB      24        0.0260  0.1611  0.1074
        96        0.0407  0.2017  0.1428
        168       0.0435  0.2085  0.1494
ES      24        0.0109  0.1044  0.0666
        96        0.0201  0.1417  0.0959
        168       0.0219  0.1480  0.1005
FR      24        0.0104  0.1019  0.0594
        96        0.0197  0.1402  0.0857
        168       0.0213  0.1458  0.0912
IT      24        0.0105  0.1022  0.0618
        96        0.0183  0.1354  0.0847
        168       0.0195  0.1397  0.0898